# Bibliotecas

In [0]:
import pandas as pd #análise de dados em tabelas
import numpy as np #análise de números
import seaborn as sb #análise em gráficos

# Diabetes Machine Learning Classification Model
With numerical variables
and Cross Validation.

Cols: 'Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'

In [0]:
import pandas as pd

In [0]:
names = ['preg', 'glic', 'pres', 'skin', 'ins', 'bmi', 'dpf', 'age', 'target']

In [0]:
d = pd.read_csv('http://bit.ly/pimadiabetes', names=names)
d.head()

,preg,glic,pres,skin,ins,bmi,dpf,age,target
0,6,148,72,35,0,33.6,627.0,50,1
1,1,85,66,29,0,26.6,351.0,31,0
2,8,183,64,0,0,23.3,672.0,32,1
3,1,89,66,23,94,28.1,167.0,21,0
4,0,137,40,35,168,43.1,2288.0,33,1


In [0]:
d.head(2)

,preg,glic,pres,skin,ins,bmi,dpf,age,target
0,6,148,72,35,0,33.6,627.0,50,1
1,1,85,66,29,0,26.6,351.0,31,0


In [0]:
X = d.drop('target', axis=1)
y = d['target']

In [0]:
from sklearn.model_selection import train_test_split
# implementing train-test-split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [0]:
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier

# random forest model creation
model = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=42)
model.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=2, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [0]:
# predictions
rfc_predict = model.predict(X_test)

In [0]:
# evaluate performance
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix

In [0]:
rfc_cv_score = cross_val_score(model, X, y, cv=10, scoring='roc_auc')

In [0]:
print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, rfc_predict))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_test, rfc_predict))
print('\n')
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())

=== Confusion Matrix ===
[[135  16]
 [ 44  36]]


=== Classification Report ===
              precision    recall  f1-score   support

           0       0.75      0.89      0.82       151
           1       0.69      0.45      0.55        80

    accuracy                           0.74       231
   macro avg       0.72      0.67      0.68       231
weighted avg       0.73      0.74      0.72       231



=== All AUC Scores ===
[0.80222222 0.80296296 0.84888889 0.7037037  0.79555556 0.87555556
 0.86       0.90222222 0.83307692 0.85153846]


=== Mean AUC Score ===
Mean AUC Score - Random Forest:  0.8275726495726496


In [0]:
# Tunning hyper parameters
from sklearn.model_selection import RandomizedSearchCV

# number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

# number of features at every split
max_features = ['auto', 'sqrt']

# max depth
max_depth = [int(x) for x in np.linspace(100, 500, num = 11)]
max_depth.append(None)

# create random grid
random_grid = {
 'n_estimators': n_estimators,
 'max_features': max_features,
 'max_depth': max_depth
 }

In [0]:
# Random search of parameters
rfc_random = RandomizedSearchCV(estimator = rfc, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

# Fit the model
rfc_random.fit(X_train, y_train)
# print results
print(rfc_random.best_params_)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   19.5s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  2.6min finished


{'n_estimators': 200, 'max_features': 'sqrt', 'max_depth': 100}


In [0]:
# After tunning
rfc = RandomForestClassifier(n_estimators=600, max_depth=300, max_features='sqrt')

# Train model with hypterparameters
rfc.fit(X_train,y_train)
rfc_predict = rfc.predict(X_test)
rfc_cv_score = cross_val_score(rfc, X, y, cv=10, scoring='roc_auc')
print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, rfc_predict))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_test, rfc_predict))
print('\n')
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())